NLtoSQL Assistant

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [2]:
!git clone --recurse-submodules https://github.com/HKUNLP/UnifiedSKG.git

fatal: destination path 'UnifiedSKG' already exists and is not an empty directory.


In [3]:
%cd UnifiedSKG

/content/drive/My Drive/UnifiedSKG


In [4]:
import sys
sys.path.append('/content/drive/My Drive/UnifiedSKG')

In [5]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 1.4 MB 53.6 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16370 sha256=28a70f21453fa558a21743dae2624b898580cf4e5672f2b128459a3e83d0884b
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [6]:
!pip install transformers==4.9.2
!pip install nltk
!pip install datasets==1.14.0
!pip install sentencepiece
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 29.4 MB/s 
     |████████████████████████████████| 3.3 MB 56.9 MB/s 
     |████████████████████████████████| 880 kB 53.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=b37b75c273961029e1a9f90ae5b7f3f1a71eaa6676ad7b43a548a76649c85422
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 290 kB 22.2 MB/s 
     |████████████████████████████████| 56 kB 2.9 MB/s 
     |████████████████████████████████| 212 kB 54.2 MB/s 
     |████████████████████████████████| 115 kB 50.8 MB/s 
  

In [7]:
import os
import time
import torch
import datasets
from googletrans import Translator 
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

In [8]:
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py',
            
            '--cfg', 'Salesforce/T5_base_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2")

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [10]:
translator = Translator()
inpt = input("Введите запрос на естественном языке:\n")
question = translator.translate(inpt, dest='en')
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = question.text
translator = Translator()
tokenized_txt = tokenizer([text_in], max_length=1024, padding="max_length", truncation=True)
pred = tokenizer.batch_decode(
    model.generate(
      torch.LongTensor(tokenized_txt.data['input_ids']),
      torch.LongTensor(tokenized_txt.data['attention_mask']),
      num_beams=1, 
      max_length=256
      ), 
    skip_special_tokens=True 
) 
print("=====SQL запрос=====")
print(*pred)


Введите запрос на естественном языке:
сколько людей в возрасте больше 36 лет?
=====SQL запрос=====
select count(*) from people where age > 36
